### Exploring Tourism information worldwide

This is a simple notebook to set up a Agent that fetches information for countries around the world across years

In [6]:
import fused
import json
import os
import time
from pathlib import Path

# Handling the OS file paths
from enum import Enum
LOCAL_OS = Enum("LOCAL_OS", ["mac", "windows"])

In [7]:
# Set your LOCAL_OS to the correct one
LOCAL_OS = LOCAL_OS.mac

if LOCAL_OS == LOCAL_OS.mac:
    PATH_TO_CLAUDE_CONFIG = (f"{str(Path.home())}/Library/Application Support/Claude/claude_desktop_config.json")
    CLAUDE_APP_PATH = "/Applications/Claude.app"
elif LOCAL_OS == LOCAL_OS.windows:
    PATH_TO_CLAUDE_CONFIG = (f"{str(Path.home())}\AppData\Roaming\Claude/claude_desktop_config.json")
    CLAUDE_APP_PATH = r"C:\Users\user\AppData\Local\AnthropicClaude\claude.exe"
    
else:
    raise ValueError("Unsupported OS")


In [8]:
# We still need your local paths

if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, "w") as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), (
    "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"
)

assert os.path.exists(CLAUDE_APP_PATH), ("Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app")

In [9]:
# Local path to the Claude app
assert os.path.exists(CLAUDE_APP_PATH), (
    "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"
)

In [13]:
# Change this path if you're not running this from the repo root
WORKING_DIR = Path.cwd().parent
print(WORKING_DIR)

/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/fused-mcp


In [14]:
# We'll load the commons folder once again to have our helper functions
commit = "5dda36c"
common = fused.load(
    f"https://github.com/fusedio/udfs/tree/{commit}/public/common"
).utils

In [15]:
# And see which agents we have available
json.load(open(os.path.join(WORKING_DIR, "agents.json")))

{'agents': [{'name': 'get_current_time', 'udfs': ['current_utc_time']},
  {'name': 'fused_docs', 'udfs': ['list_public_udfs', 'reading_fused_docs']},
  {'name': 'vancouver_open_data',
   'udfs': ['hundred_parks_in_vancouver', 'internet_speeds_for_lat_lon']},
  {'name': 'elevation_stats_for_lat_lon_area', 'udfs': ['elevation_stats']},
  {'name': 'dynamic_output_vector', 'udfs': ['dynamic_output_vector_udf']},
  {'name': 'vancouver_open_data_demo',
   'udfs': ['hundred_parks_in_vancouver',
    'electric_vehicle_chargers_in_vancouver',
    'yearly_crime_amount',
    'internet_speeds_for_lat_lon',
    'community_gardens_vancouver']},
  {'name': 'world_countries_info', 'udfs': ['get_country_data']},
  {'name': 'routing_agent', 'udfs': ['single_routing_agent']},
  {'name': 'tourism_info', 'udfs': ['visitors_arrival', 'visitors_purpose']},
  {'name': 'stock_information', 'udfs': ['get_stock_details']}]}

In [16]:
AGENT_NAME = "tourism_info"

In [17]:
@fused.udf
def get_visitor_data(year: int = None, country: str = None):
    """
    UDF to retrieve visitor arrival data with optional filters.
    We've simply hosted a CSV file on Google Drive for the sake of simplicity for this example.

    Parameters:
    - year (int): Filter by specific year (e.g., 2023)
    - country (str): Filter by specific country name (e.g., 'Thailand')

    Returns:
    A DataFrame with the following columns:
    - Country
    - Purpose_of_visit_to_Japan
    - Year
    - Growth Rate(%)
    - Visitor Arrivals
    """
    import pandas as pd
    import fused
    from io import BytesIO
    import requests

    drive_url = "https://drive.google.com/uc?export=download&id=1OQkx2fLA6oQ3fzNL-aiCFqDr65bp-nAL"

    @fused.cache
    def load_csv():
        response = requests.get(drive_url)
        response.raise_for_status()
        return pd.read_csv(BytesIO(response.content), encoding='utf-8')

    df = load_csv()
    df.rename(columns={"Country/Area": "Country"}, inplace=True)

    # Cleaning up
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
    if year is not None:
        df = df[df['Year'] == year]
    if country:
        df = df[df['Country'] == country]

    # Columns to return
    columns = [
        'Country',
        'Purpose_of_visit_to_Japan',
        'Year',
        'Growth Rate(%)',
        'Visitor Arrivals'
    ]

    return df[columns] if not df.empty else pd.DataFrame(columns=columns)


In [18]:
fused.run(get_visitor_data)

UDF result was cached.

,Country,Purpose_of_visit_to_Japan,Year,Growth Rate(%),Visitor Arrivals
0,China,Tourism,1990,None,None
1,China,Tourism,1991,None,None
2,China,Tourism,1992,None,"29,147"
3,China,Tourism,1993,-9.239372834,"26,454"
4,China,Tourism,1994,-7.197399259,"24,550"
...,...,...,...,...,...
1045,United States,Business,2020,-86.664479278,"28,857"
1046,United States,Business,2021,-95.515819385,"1,294"
1047,United States,Business,2022,"4,408.191653787","58,336"
1048,United States,Business,2023,104.588933077,"119,349"


In [19]:
get_visitor_data_metadata = {
    "description": """
Name: get_visitor_data
Purpose and Functionality:
The UDF 'get_visitor_data' provides detailed visitor arrival statistics from a structured dataset sourced from a CSV file hosted on Google Drive. It allows optional filtering by year and country, and returns comprehensive information including purpose of visit, growth rate, and number of visitor arrivals.

Input Parameters:
- year (int, optional): The year for which detailed visitor data is requested. If not specified, data for all available years will be returned.
- country (str, optional): The country for which visitor data is requested. If not specified, data for all countries will be returned.

Output:
Returns a Pandas DataFrame with the following columns:
- Country/Area
- Purpose_of_visit_to_Japan
- Year
- Growth Rate(%)
- Visitor Arrivals

If no matching records are found, an empty DataFrame with the above column structure is returned.

Technical Details and Limitations:
- The CSV is fetched from a public Google Drive link using HTTP requests. Internet access is required.
- The 'Year' column is cast to integer for filtering.
- The function gracefully handles missing or empty fields, such as blank growth rates or visitor counts.
- If the source format of the CSV changes, the behavior may be affected.
    """,
    "parameters": [
        {
            "name": "year",
            "type": "integer",
            "default": None,
            "description": "The year to filter visitor data. Optional."
        },
        {
            "name": "country",
            "type": "string",
            "default": None,
            "description": "The country to filter visitor data. Optional."
        }
    ]
}
get_visitor_data_metadata

{'description': "\nName: get_visitor_data\nPurpose and Functionality:\nThe UDF 'get_visitor_data' provides detailed visitor arrival statistics from a structured dataset sourced from a CSV file hosted on Google Drive. It allows optional filtering by year and country, and returns comprehensive information including purpose of visit, growth rate, and number of visitor arrivals.\n\nInput Parameters:\n- year (int, optional): The year for which detailed visitor data is requested. If not specified, data for all available years will be returned.\n- country (str, optional): The country for which visitor data is requested. If not specified, data for all countries will be returned.\n\nOutput:\nReturns a Pandas DataFrame with the following columns:\n- Country/Area\n- Purpose_of_visit_to_Japan\n- Year\n- Growth Rate(%)\n- Visitor Arrivals\n\nIf no matching records are found, an empty DataFrame with the above column structure is returned.\n\nTechnical Details and Limitations:\n- The CSV is fetched f

In [24]:
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=get_visitor_data,
    agent_name=AGENT_NAME,
    udf_name="retrieve_visitor_data",
    mcp_metadata=get_visitor_data_metadata,
)

In [25]:
# Let's make sure we created our agent properly, with all our UDFs
agents = json.load(open(os.path.join(WORKING_DIR, "agents.json")))
print(json.dumps(agents, indent=4, sort_keys=True))

{
    "agents": [
        {
            "name": "get_current_time",
            "udfs": [
                "current_utc_time"
            ]
        },
        {
            "name": "fused_docs",
            "udfs": [
                "list_public_udfs",
                "reading_fused_docs"
            ]
        },
        {
            "name": "vancouver_open_data",
            "udfs": [
                "hundred_parks_in_vancouver",
                "internet_speeds_for_lat_lon"
            ]
        },
        {
            "name": "elevation_stats_for_lat_lon_area",
            "udfs": [
                "elevation_stats"
            ]
        },
        {
            "name": "dynamic_output_vector",
            "udfs": [
                "dynamic_output_vector_udf"
            ]
        },
        {
            "name": "vancouver_open_data_demo",
            "udfs": [
                "hundred_parks_in_vancouver",
                "electric_vehicle_chargers_in_vancouver",
                "

In [22]:
AGENT_NAME

'tourism_info'

In [23]:
# Finally, we can select which Agent we want to pass to Claude in our MCP server config
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list=[AGENT_NAME],
    repo_path=WORKING_DIR,
)

In [87]:
# Let's read this Claude Desktop config to see what we're passing
claude_config = json.load(open(PATH_TO_CLAUDE_CONFIG))
print(json.dumps(claude_config, indent=4, sort_keys=True))

{
    "mcpServers": {
        "tourism_info": {
            "args": [
                "run",
                "--directory",
                "C:\\Users\\user\\fused-mcp",
                "main.py",
                "--runtime=local",
                "--udf-names=visitors_arrival,visitors_purpose",
                "--name=tourism_info"
            ],
            "command": "uv"
        }
    }
}


In [ ]:
# Restarting Claude
def restart_claude(claude_path: str = CLAUDE_APP_PATH):
    app_name = claude_path.split("/")[-1]

    try:
        os.system(f"pkill -f '{app_name}'")
        print(f"Killed {app_name}")
        time.sleep(2)  # Wait for shutdown
    except Exception:
        print("Claude wasn't running, so no need to kill it")

    print(f"Restarting {app_name}")
    os.system(f"open -a '{claude_path}'")  # Restart Claude

restart_claude()